In [ ]:
import torch
import copy
import torch.nn.functional as F
from models.tiny_vgg import TinyVGG
from models.module import QLinear,QConv2d
def cmp(a,b):
    return (a-b).abs().max()
q_b=8
model = TinyVGG(image_size=32,num_classes=10)
model.eval()
model.fuse_bn()
model.quantize(w_num_bits=q_b,a_num_bits=q_b)
model.load_state_dict(torch.load(f'./qat_logs/cifar10/tiny_vgg_w{q_b}a{q_b}/checkpoint_max.pth')['net'])
q_model = copy.deepcopy(model)
q_model.freeze()


q_model.stage1.conv1.conv_module.weight

In [2]:
q_model.stage1.conv0.conv_module.weight.max()

tensor(110., grad_fn=<MaxBackward1>)

In [16]:
dummy_input = torch.randn(1, 3, 32, 32)

cmp(model.stage1.conv0(dummy_input), q_model.stage1.conv0(dummy_input))
s1c0=F.relu(model.stage1.conv0(dummy_input))

q_s1c0=q_model.stage1.conv0.qo.quantize_tensor(s1c0)

s1c1 = (model.stage1.conv1(s1c0))
q_s1c1 = (q_model.stage1.conv1.quantize_inference(q_s1c0))
q_s1c1 = q_model.stage1.conv1.qo.dequantize_tensor(q_s1c1)
cmp(s1c1,q_s1c1)

tensor(0.8249, grad_fn=<MaxBackward1>)

In [14]:
cmp(q_model.stage1.conv0.qo.dequantize_tensor(q_s1c0),s1c0)

tensor(0., grad_fn=<MaxBackward1>)

In [15]:
q_s1c0

tensor([[[[ 54.,   5.,   0.,  ...,   0.,  25.,   0.],
          [ 41.,   0.,   0.,  ...,  14.,   0.,  20.],
          [  0.,   0.,  19.,  ...,   0.,  78.,   0.],
          ...,
          [ 25.,   0.,  45.,  ...,  80.,   0.,  42.],
          [  0.,   0.,  42.,  ...,   0.,   6.,  27.],
          [  7.,   0.,  70.,  ...,  35.,   0.,  81.]],

         [[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          ...,
          [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

         [[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          ...,
          [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          [  0.,   0.,   0.,  ...

In [5]:
q_out

tensor([[-7.6420, -6.6868, -5.7315, -7.6420, -7.6420, -6.6868, -7.6420, -7.6420,
         -7.6420, -7.6420]], grad_fn=<MulBackward0>)

In [14]:
q_model.linear.qo

QParam()

In [5]:
q_model.layer1[0].conv1.n

tensor(10.)